# Домашнее задание 3 [10 баллов] 
# До 16.05.18 23:59

Задание выполняется в группе (1-4 человека). В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.

Не все части обязательны для выполнения, однако вы можете быть дополнительно оштрафованы за небрежное за выполнение одной или двух частей вместо четырех.

При возниконовении проблем с выполнением задания обращайтесь с вопросами к преподавателю. Поэтому настоятельно рекомендуется выполнять задание заранее, оставив запас времени на всевозможные технические проблемы. Если вы начали читать условие в последний вечер и не успели из-за проблем с установкой какой-либо библиотеки — это ваши проблемы.


Результат выполнения задания — это отчёт в формате html на основе Jupyter Notebook. Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание **минимума** необходимой теории и/или описание используемых инструментов - не стоит переписывать лекции или Википедию
* Подробный пошаговый рассказ о проделанной работе
* Аккуратно оформленные результаты
* **Внятные выводы** – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Небрежное его оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в тексте в явном виде.

Сдача отчетов осуществляется через систему AnyTask.



# Классификация текстов с активным обучением


Зададимся простой задачей классификации текстов: например, классификацией отзывов на банки по тональности. Эта задача решается с достаточно высокими показателями качества с использованием стандартных алгоритмов классификации, например, сверточных нейронных сетей: корпус состоит из достаточного количества документов, чтобы сверточная сеть хорошо обучилась. Однако возникает естественный вопрос: действительно ли все документы нужны для того, чтобы достичь таких высоких показателей качества (или сопоставимых с ними). Парадигма активного обучения поможет вам ответить на этот вопрос.

## Часть 1. Предобработка данных [2 балла]

Коллекция отзывов хранится в файле banki_responses (https://www.dropbox.com/s/ol3ux3ibr6rd5ke/banki_responses.json.bz2?dl=0). Одна строчка в этом файле соответствует одному json-словарю. Из этого словаря вам понадобятся два значения по ключам text и rating -- текст отзыва и его оценка по шкале от 1 до 5.   

Считайте файл. Посчитайте, каких отзывов больше: положительных или отрицательных? 

Проведите предварительную обработку данных: удалите слишком короткие и слишком длинные тексты (пороги на длину определите самостоятельно). 

Разбейте данные на обучающее ($train$) и тестовое ($test$) множество случайным образом в отношеннии 3:1 (или любом другом отношении, которое покажется вам разумным). 

Задача классификации сформулирована так: по каждому отзыву определить его оценку (т.е. классификация на 5 классов). Признаками для классификации выступают слова.

In [3]:
import json

responses = []
for line in open('banki_responses.json', 'r'):
    responses.append(json.loads(line))

In [6]:
key_text = 'text'
key_grade = 'rating_grade'

In [11]:
from collections import Counter

grade_distribution = Counter(map(lambda record: record[key_grade], responses))

Как видим негативные отзывы превалируют над положительными:

In [12]:
grade_distribution

Counter({1: 72307, 2: 19928, 3: 11702, 4: 5537, 5: 26715, None: 64841})

In [81]:
responses = list(filter(lambda x: x[key_grade] != None, responses))

In [ ]:
import re

def word_count(string):
    return len(re.findall("\w+", string))

In [ ]:
responses_lengths = list(map(lambda record: word_count(record[key_text]), responses))
responses_lengths_distribution = Counter(responses_lengths)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline  

zipped_length_distribution = list(zip(*length_distribution.items()))

plt.figure(figsize=(20,10))
plt.bar(zipped_length_distribution[0], zipped_length_distribution[1])
plt.show()

In [88]:
def count_part_in_range(left, right):
    return len(list(x for x in responses_lengths if x >= left and x <= right)) / len(responses_lengths)

Предварительно возьмем такой диапазон длин. На их долю приходится 86% выборки.

In [89]:
left_length_border = 40
right_length_border = 400

In [90]:
count_part_in_range(left_length_border, right_length_border)

0.8616628362055673

In [241]:
x, y = [resp[key_text] for resp in responses], [resp[key_grade] for resp in responses]

In [242]:
from random import shuffle

zipped_data = list(zip(x, y))
shuffle(zipped_data)

x, y = list(zip(*zipped_data))

In [243]:
train_part_coeff = 0.8
train_part_size = int(len(x) * train_part_coeff)

## Часть 2. Baseline [4 балла]

Получите baseline классификации: в идеале, используйте сверточную нейронную сеть (слой эмбеддингов + свертка + субдескритизация). Число и размерность фильтров определите самостоятельно, так же как и использование регуляризаторов (dropout / batch norm) и их параметров. Так же самостоятельно (но обосновано) решите, использовать ли вам предобученные эмбеддинги или нет и проводить ли вам лемматизацию или нет. 

Обучите сеть на обучающем множестве и протестируйте на тестовом. Зафиксируйте baseline.

Если совсем трудно или вычисления занимают слишком много времени, используйте любой другой известный и симпатичный вам алгоритм классификации. 

Используем ембеддинги из 2-ого задания

In [117]:
import numpy as np

embeddings = {}
with open('/Users/user/MIPT-text-analysis/wiki.ru.vec', 'r') as fin:
    word_count, embedding_size = [int(number) for number in fin.readline().split(' ')]
    for i in range(word_count):
        data = fin.readline().split(' ')[:-1]
        word = data[0]
        embedding = np.array([float(number) for number in data[1:]])
        embeddings[word.lower()] = embedding

In [272]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [245]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

In [250]:
word_index = tokenizer.word_index

In [286]:
data = pad_sequences(sequences, maxlen=right_length_border)
labels = to_categorical(np.asarray(y) - 1)

In [287]:
x_train = data[:train_part_size]
y_train = labels[:train_part_size]

x_test = data[train_part_size:]
y_test = labels[train_part_size:]

In [275]:
EMBEDDING_DIM = len(embeddings['мама'])

In [276]:
word_index

{'11ти': 261678,
 'бельденкову': 287242,
 '5839': 164105,
 'собственным': 10354,
 'министратегии': 221839,
 'недополученные': 61478,
 'райффайзнбанка': 334319,
 '89191080': 271199,
 'я\xa0посмотрел': 225997,
 'мороза»': 238762,
 'обусловил': 91346,
 'предъявляешь': 98648,
 'высокоточный': 237441,
 'подытожили': 223935,
 'компромиссный': 56511,
 'отсудил': 73886,
 'хелицерами': 173250,
 'уставая': 115055,
 'огорожены': 93243,
 'исключило': 111040,
 "вась''": 204491,
 'выдворения': 281242,
 'ламинатная': 236770,
 'непринято': 114707,
 'растекания': 234854,
 'егэ': 56385,
 'усмешек': 150296,
 'плачут': 43511,
 'ограниченности': 44847,
 '045385': 335842,
 '«девочка»': 76305,
 'спрашивалось': 63506,
 'корки': 35831,
 'хлебавши”': 269140,
 'бьется»': 249120,
 '84753': 205981,
 'балаганом': 157992,
 'пружинить': 293693,
 'докатились': 74697,
 'забирать»': 167072,
 '8490': 306544,
 '933': 51628,
 'последений': 179636,
 'винтики': 99550,
 '«производственной»': 276174,
 '11298330': 257323,
 'ашк

In [277]:
embeddings_matrix = np.zeros((len(word_index)+1, EMBEDDING_DIM), dtype=np.float32)

for word, word_id in word_index.items():
    if word in embeddings:
        embeddings_matrix[word_id] = embeddings[word]

In [278]:
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, GlobalMaxPooling1D
from keras import Model

In [279]:
sequence_input = Input(shape=(right_length_border,), dtype='int32')
embedding_layer = Embedding(len(word_index)+1,
                            EMBEDDING_DIM,
                            weights=[embeddings_matrix],
                            input_length=right_length_border,
                            trainable=False)
embedded_sequences = embedding_layer(sequence_input)

In [280]:
net = Conv1D(128, 5, activation='relu')(embedded_sequences)
net = MaxPooling1D(5)(net)
net = Conv1D(128, 5, activation='relu')(net)
net = GlobalMaxPooling1D()(net)
net = Dense(64, activation='relu')(net)
preds = Dense(5, activation='softmax')(net)

In [289]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test),
          epochs=2, batch_size=128)

Train on 108951 samples, validate on 27238 samples
Epoch 1/2
 27392/108951 [======>.......................] - ETA: 21:29 - loss: 1.0893 - acc: 0.6202

## Часть 3. Baseline [4 балла] Активное обучение

Подход активного обучения основан на следующей идее: вместо всего обучающего множества мы используем его маленькие фрагменты, в которых модель неуверена для обучения. Таким образом, модель обучается исключительно по **трудным** объектам, число которых существенно меньше, чем общее число объектов.

Обучение модели начинается с обучения по $N$ случайно выбранным примерам, где $N$ – небольшое число (100, 200 и т.д.). Затем модель тестируется на $|train| - N$ объектах, после чего из  $|train| - N$ объектов выбираются снова $N$  объектов, в которых модель не уверена. Эти объекты используются для дообучения модели. Процесс выбора $N$ трудных объектов и дообучения на них повторяется некоторое количество раз (100, 200 и т.д. раз). На каждом шаге активного обучения модель можно протестировать на тестовом множестве, чтобы сравнить ее качества с baseline.

Как выбирать трудные объекты:
1. Выход нейронной сети - оценки 5 вероятностей принадлежности объекта одному из классов. Предсказанный класс – это тот класс, вероятность которого максимальна. Отсортируем объекты по убыванию вероятности предсказанного класса ($\min \max p_i$) и выберем $N$ первых объектов;
2. Используем энтропию: чем больше энтропия предсказания, тем ближе распределение вероятностей предсказания к равномерному распределению, тем труднее объект. 